In [1]:
from Lib.RPS_game import DELETE_ABBEY, play, mrugesh, abbey, quincy, kris, human, random_player
from Lib.RPS_encoding import RPS_encode, RPS_decode
from Lib.Telepathic_player import telepath
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, LSTM, Embedding
import pandas as pd
from os import listdir
from os.path import isfile, join
import random
import itertools
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
import numpy as np

## Parameters

In [2]:
game_len = 1000
batch_size = 100
order = 2
keys = ['0', '1', '2']

# Data Preparation
## Reading Training Data

In [3]:
path = "Games/"
all_game_paths = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
games_concat = pd.DataFrame(columns = ["Player","Opponent","Result","Winning_Play"])
games = []
for game_path in all_game_paths:
    #print(game_path)
    game_data = pd.read_csv(game_path, names = ["Player","Opponent","Result","Winning_Play"])
    games.append(game_data)
    games_concat = games_concat.append(game_data)
    
#print(len(game_data.index)) 
#game_data.head()  
print(len(games_concat.index))
games_concat.head()

120000


,Player,Opponent,Result,Winning_Play
0,P,P,0,S
1,P,P,0,S
2,R,S,1,R
3,P,P,0,S
4,S,P,1,S


## Feature Engineering
Calculate the freqeuncy of past plays from each state and prepare for input

In [4]:
def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return itertools.zip_longest(*args, fillvalue=fillvalue)

def create_states(count):            
    states = []
    for i in itertools.product(keys, repeat=count):
        states.append(''.join(i))
    return states

def get_dist(array, order):
    patterns = create_states(order)
    dist = {pat : 0 for pat in patterns}
    rolling_pattern = patterns[0]
    for play in array:
        rolling_pattern = rolling_pattern[1:] + str(play)
        dist[rolling_pattern] += 1
    return dist

def normalise_dist(dist):
    #look at all possible next states from each pattern and group
    #for each group subtract minimum count from total 
    groups = [[''.join(pat) + key for key in keys] for pat in itertools.product(keys, repeat=order-1)]
    for group in groups:
        counts = [dist[key] for key in group]
        #Normalize group between 0-1
        for key in group:
            if(max(counts)==min(counts)):
                dist[key] = 0
            else:
                dist[key] = (dist[key]-min(counts))/(max(counts)-min(counts))            
    
    return list(dist.values())

def get_all_normalised_dist(array, order):
    all_dist = []
    patterns = create_states(order)
    dist = {pat : 0 for pat in patterns}
    rolling_pattern = patterns[0]
    for play in array:
        rolling_pattern = rolling_pattern[1:] + str(play)
        dist[rolling_pattern] += 1
        all_dist.append(list(normalise_dist(dist)))
    return all_dist

def prep_input(prev_play, dist):
    return [prev_play] + dist
    
temp = games_concat.drop("Result", axis = 1)
temp.index = [x for x in range(temp.index.size)]

#handling feature engineering in pandas
player_input = [RPS_encode(x) for x in temp.loc[:,"Player"]]
winning_plays = [RPS_encode(x) for x in temp.loc[:,"Winning_Play"]]
#past player play distribution:


patterns = create_states(order)
all_dists = []
#-every 1000 entries reset distribution (simulates multiple games)
for game in grouper(player_input, game_len, fillvalue=0):
    all_dists += get_all_normalised_dist(game, order)
        
print(len(all_dists))
#print(all_dists[0:5])
#print(all_dists[998:1003])  
input_dataframe = pd.DataFrame(all_dists, columns = patterns)
input_dataframe = pd.concat([pd.get_dummies(player_input, prefix = 'Prev_Play'), input_dataframe], axis=1)
#input_dataframe.drop(input_dataframe.tail(1).index, inplace=True)
print(input_dataframe)

target_series = pd.Series(winning_plays)
#target_series.drop(target_series.head(1).index, inplace=True)
#target_series.index = [x for x in range(len(input_dataframe))]
print(target_series)

    
#tempInput = [ for prev_play, dist in zip(player_input, all_dists)]
#tempTarget = [RPS_encode(x) for x in temp.loc[:,"Winning_Play"]]

120000
        Prev_Play_0  Prev_Play_1  Prev_Play_2   00   01   02   10   11   12  \
0                 0            1            0  0.0  1.0  0.0  0.0  0.0  0.0   
1                 0            1            0  0.0  1.0  0.0  0.0  1.0  0.0   
2                 1            0            0  0.0  1.0  0.0  1.0  1.0  0.0   
3                 0            1            0  0.0  1.0  0.0  1.0  1.0  0.0   
4                 0            0            1  0.0  1.0  0.0  0.0  0.0  0.0   
...             ...          ...          ...  ...  ...  ...  ...  ...  ...   
119995            0            0            1  0.0  0.0  0.0  1.0  1.0  0.0   
119996            0            0            1  0.0  0.0  0.0  1.0  1.0  0.0   
119997            0            1            0  0.0  0.0  0.0  1.0  1.0  0.0   
119998            0            0            1  0.0  0.0  0.0  0.0  0.0  0.0   
119999            1            0            0  0.0  0.0  0.0  0.0  0.0  0.0   

         20   21   22  
0       0.0  0.0  0.

Convert pandas DataFrame to tensorflow Dataset, batch inputs and offset targets and inputs so the input predicts the next target. 

In [5]:
input_data = tf.data.Dataset.from_tensor_slices(input_dataframe)
target_data = tf.data.Dataset.from_tensor_slices(target_series)


#split into training batches
input_batches = input_data.batch(batch_size+1, drop_remainder = True)
target_batches = target_data.batch(batch_size+1, drop_remainder = True)

#When predicting based off player inputs need to shift targets one time step
#remove last element from all input sequences
def remove_last(chunk):
    return chunk[:-1]
input_batches = input_batches.map(remove_last)

#remove first element from all target sequences
def remove_first(chunk):
    return chunk[1:]
target_batches = target_batches.map(remove_first)

Combine input and target batches then split the dataset into training, test and validation sets

In [6]:
full_dataset = tf.data.Dataset.zip((input_batches, target_batches)).shuffle(10000)

dataset_size = len(full_dataset)
train_size = int(0.7*dataset_size)
test_size = int(0.15*dataset_size)
val_size = int(0.15*dataset_size)

train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.take(test_size)
test_dataset = test_dataset.skip(test_size)

print("Total sequence count: " + str(dataset_size))
print("Training set: " + str(len(train_dataset)))
print("Test set: " + str(len(test_dataset)))
print("Validation set: " + str(len(val_dataset)))
for x , y in train_dataset.take(1):
    print(x.numpy()[0:5])
    print(y.numpy()[0:5])


Total sequence count: 1188
Training set: 831
Test set: 179
Validation set: 178
[[0.         0.         1.         0.         1.         0.5
  0.         0.55555556 1.         0.         1.         1.        ]
 [0.         0.         1.         0.         1.         0.5
  0.         0.55555556 1.         0.         0.5        1.        ]
 [0.         1.         0.         0.         1.         0.5
  0.         0.55555556 1.         0.         1.         0.66666667]
 [0.         1.         0.         0.         1.         0.5
  0.         1.         0.64285714 0.         1.         0.66666667]
 [0.         0.         1.         0.         1.         0.5
  0.         0.60869565 1.         0.         1.         0.66666667]]
[1 1 1 1 1]


# Build the model

In [7]:
def build_model(input_dims, vocab_size, batch_size):
    model = keras.Sequential()
    model.add(keras.layers.InputLayer(batch_input_shape = [batch_size, input_dims]))
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(12, activation = 'relu'))
    model.add(Dense(vocab_size, activation = 'softmax'))
    return model
vocab_size = len(keys)
model = build_model(len(patterns)+vocab_size, vocab_size, batch_size)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (100, 64)                 832       
_________________________________________________________________
dense_1 (Dense)              (100, 12)                 780       
_________________________________________________________________
dense_2 (Dense)              (100, 3)                  39        
Total params: 1,651
Trainable params: 1,651
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

In [9]:
print(train_dataset)

<TakeDataset shapes: ((100, 12), (100,)), types: (tf.float64, tf.int64)>


# Train the model

In [10]:
model.fit(train_dataset, epochs = 50)

Epoch 1/50
831/831 [==============================] - 3s 2ms/step - loss: 0.7962 - accuracy: 0.6539
Epoch 2/50
831/831 [==============================] - 2s 2ms/step - loss: 0.6449 - accuracy: 0.7337
Epoch 3/50
831/831 [==============================] - 2s 2ms/step - loss: 0.6145 - accuracy: 0.7430
Epoch 4/50
831/831 [==============================] - 2s 2ms/step - loss: 0.6007 - accuracy: 0.7416
Epoch 5/50
831/831 [==============================] - 2s 2ms/step - loss: 0.5880 - accuracy: 0.7478
Epoch 6/50
831/831 [==============================] - 2s 2ms/step - loss: 0.5776 - accuracy: 0.7484
Epoch 7/50
831/831 [==============================] - 2s 2ms/step - loss: 0.5641 - accuracy: 0.7507
Epoch 8/50
831/831 [==============================] - 2s 2ms/step - loss: 0.5702 - accuracy: 0.7503
Epoch 9/50
831/831 [==============================] - 2s 2ms/step - loss: 0.5544 - accuracy: 0.7554
Epoch 10/50
831/831 [==============================] - 2s 2ms/step - loss: 0.5653 - accuracy: 0.7497

In [11]:
test_loss, test_acc = model.evaluate(test_dataset,verbose = 1)
print('Test accuracy: ' + str(test_acc))


179/179 [==============================] - 0s 2ms/step - loss: 0.5032 - accuracy: 0.7711
Test accuracy: 0.7710614800453186


In [12]:
testing_input = [0 for x in range(len(patterns)+vocab_size)]
testing_input[0] = 1
model.predict([testing_input])

array([[0.28083214, 0.39425772, 0.32491016]], dtype=float32)

# Save or Load model

In [20]:
model.save('Models/DNN_PlayerPastInputFrequency_Abbey')

INFO:tensorflow:Assets written to: Models/DNN_PlayerPastInputFrequency\assets


In [7]:
model = keras.models.load_model('Models/DNN_PlayerPastInputFrequency_Abbey') 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (100, 64)                 832       
_________________________________________________________________
dense_1 (Dense)              (100, 12)                 780       
_________________________________________________________________
dense_2 (Dense)              (100, 3)                  39        
Total params: 1,651
Trainable params: 1,651
Non-trainable params: 0
_________________________________________________________________


# Testing the model

Define functions to interface between the game and the model

In [8]:
def Get_Prediction(model, player_history):
    #format history for input
    input_eval = [RPS_encode(c) for c in player_history]
    input_eval = get_dist(input_eval, order)
    input_eval = normalise_dist(input_eval)
    RPS = {'R':[1,0,0],'P':[0,1,0],'S':[0,0,1]}
    input_eval = RPS[player_history[-1]] + input_eval
    
    #print(input_eval)
    
    input_eval = tf.expand_dims(input_eval, 0)
    
    #high temperature  => suprising output
    #low temperature => predictable output
    temperature = 0.1; 
    
    model.reset_states()
    
    prediction = model(input_eval)
    #prediction = tf.squeeze(prediction, 0)
    
    prediction = prediction / temperature
    #print(prediction)
    predicted_id = tf.random.categorical(prediction, num_samples = 1)[-1,0].numpy()
    #print(predicted_id)
    #print("Guessing: " + RPSpair_decode(predicted_id))
    return RPS_decode(predicted_id)[0]

In [9]:
plays = 1000

def DELETE_PLAYER():
    player('', isClear = True)

def player(prev_play, opponent_history = [], play_history=['R'], isClear = False):
    if isClear: 
        opponent_history.clear()
        play_history.clear()
        play_history.append('R')
        return
    
    opponent_history.append(prev_play)
    #print("Actual: " + prev_play) 
    guess = Get_Prediction(model, play_history)
    #print(guess)
    play_history.append(guess)
    return guess

## Play RPS games
Clears player's and abbey's memory before each run

In [10]:
DELETE_PLAYER()
DELETE_ABBEY()
play(player, abbey, plays)
DELETE_PLAYER()
DELETE_ABBEY()
play(player, abbey, plays)
DELETE_PLAYER()
DELETE_ABBEY()
play(player, abbey, plays)
DELETE_PLAYER()
DELETE_ABBEY()
play(player, abbey, plays)

Final results: {'p1': 785, 'p2': 61, 'tie': 154}
Player 1 win rate: 92.78959810874704%
Final results: {'p1': 723, 'p2': 112, 'tie': 165}
Player 1 win rate: 86.58682634730539%
Final results: {'p1': 714, 'p2': 112, 'tie': 174}
Player 1 win rate: 86.4406779661017%
Final results: {'p1': 720, 'p2': 111, 'tie': 169}
Player 1 win rate: 86.64259927797833%


86.64259927797833